# Options Pricing

A note: this notebook was/is used by the author to learn about options pricing and implementing it in python. as such, there are a lot of comments which are just memory aids for me when I come back to things. Final versions of functions (the .py files (see GitHub)) are much more cleanly formatted and commented.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sp
from scipy.stats import norm
import matplotlib.pyplot as plt
from stockdex import Ticker

## Stock Details

In [7]:
def volatility(stock):
    """Calculates the volatility of a stock, pulling 1 month of data from Yahoo Finance, with a 1d granularity"""
    if type(stock) != str:
        print("Please enter a valid stock code")
        return None
    ticker = Ticker(ticker=stock)
    price = ticker.yahoo_api_price(range='1mo', dataGranularity='1d')
    price_arr = price['close']
    pct_chgs = price_arr.pct_change().dropna()  # Calculate % changes 
    std = np.std(pct_chgs)
    volatility = std*np.sqrt(252) #Annualized volatility from 1mo of data.
    return volatility

In [6]:
#AAPL
stock = 'AAPL'
#ticker = Ticker(ticker=stock)
#S0 = ticker.yahoo_api_price(range='1d', dataGranularity='1d')['close'].values[0] #initial price
S0 = 201.56 #2/6/25 price
K = 120 #strike price
T = 1 #Time to maturity in years
r = 0.04258 #risk free rate: us treasury 10 year bond yield 22/3/25
#sigma = volatility(stock) #volatility
sigma = 0.33
sigmas = np.linspace(0,1,11)
Ks = np.linspace(80,120,5)
S0,sigma, sigmas,Ks

(201.56,
 0.33,
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 array([ 80.,  90., 100., 110., 120.]))

## Black Scholes

In [29]:
def black_scholes_call(S0,K,T,r,sigma):
    """Calculates call price of an European Option using the Black-Scholes Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility"""
    
    # At or past maturity
    if T <= 0:
        return max(S0 - K, 0.0)

    # Zero volatility -> option is equivalent to a forward payoff discounted
    if sigma <= 0:
        return max(S0 - K * np.exp(-r * T), 0.0)

    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - (sigma*np.sqrt(T))
    
    price = S0*sp.norm.cdf(d1) - K*np.exp(-r*T)*sp.norm.cdf(d2)
    
    return price

In [27]:
def black_scholes_put(S0,K,T,r,sigma):
    """Calculates put price of an European Option using the Black-Scholes Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility"""
    
    # At or past maturity
    if T <= 0:
        return max(K - S0, 0.0)

    # Zero volatility -> option is equivalent to a forward payoff discounted
    if sigma <= 0:
        return max(K * np.exp(-r * T) - S0, 0.0)
    
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - (sigma*np.sqrt(T))
    
    price = K*np.exp(-r*T)*sp.norm.cdf(-d2) - S0*sp.norm.cdf(-d1)
    return price

In [67]:
for s in sigmas:
    price = black_scholes_call(S0,K,T,r,s)
    print(f'Volatility: {s:.1f}\n Risk Neutral Price:{price:.4f}')

Volatility: 0.0
 Risk Neutral Price:86.5623
Volatility: 0.1
 Risk Neutral Price:86.5623
Volatility: 0.2
 Risk Neutral Price:86.5850
Volatility: 0.3
 Risk Neutral Price:87.1024
Volatility: 0.4
 Risk Neutral Price:88.7498
Volatility: 0.5
 Risk Neutral Price:91.4574
Volatility: 0.6
 Risk Neutral Price:94.9217
Volatility: 0.7
 Risk Neutral Price:98.8830
Volatility: 0.8
 Risk Neutral Price:103.1568
Volatility: 0.9
 Risk Neutral Price:107.6157
Volatility: 1.0
 Risk Neutral Price:112.1702


In [68]:
for K in Ks:
    price = black_scholes_call(S0,K,T,r,sigma)
    print(f'Strike Price: {K:.1f}\n Risk Neutral Price: {price:.4f} \n {sigma}')

Strike Price: 80.0
 Risk Neutral Price: 124.9147 
 0.33
Strike Price: 90.0
 Risk Neutral Price: 115.3805 
 0.33
Strike Price: 100.0
 Risk Neutral Price: 105.9190 
 0.33
Strike Price: 110.0
 Risk Neutral Price: 96.5900 
 0.33
Strike Price: 120.0
 Risk Neutral Price: 87.4706 
 0.33


## Binomial Tree

In [19]:
def binomial_tree_call(S0,K,T,r,sigma,N=10000):
    """Calculates call price of an European Option using the Binomial Tree Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility
    N = Number of steps within time T"""
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    p_u = (np.exp(r*dt)-d)/(u-d)
    p_d = 1-p_u
    
    #empty array for price tree
    prices = np.zeros((N+1, N+1))
    for i in range(N+1): #Step number
        for j in range(i+1): #Node within that step
            prices[j,i] = S0 * (u**j) * (d**(i-j)) #j and i-j so they count in opposite directions (one up to i+1, one down from i to 0)
                                                    #All down, to all up.
    #Finds all possible payoffs
    option_values = np.maximum(prices[:, N]-K,0) #if price - K is less than 0, sets payoff to zero. (wouldn't use option)
    
    #loop backwards from final price to find best price for now
    for k in range(N-1, -1, -1):
        for l in range(k+1): #possible nodes at step l
            #https://users.physics.ox.ac.uk/~Foot/Phynance/Binomial2013.pdf
            # exp term accounts for risk free return. Rest is standard expected value calculation.
            option_values[l] = np.exp(-r*dt)*((p_u*option_values[l])+((p_d)*option_values[l+1]))
            
    return option_values[0] #returns value at node 0 (the start, ie. how you should price it)

In [20]:
def binomial_tree_put(S0,K,T,r,sigma,N=10000):
    """Calculates put price of an European Option using the Binomial Tree Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility
    N = Number of steps within time T"""
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    p_u = (np.exp(r*dt)-d)/(u-d)
    p_d = 1-p_u
    
    #empty array for price tree
    prices = np.zeros((N+1, N+1))
    for i in range(N+1): #Step number
        for j in range(i+1): #Node within that step
            prices[j,i] = S0 * (u**j) * (d**(i-j)) #j and i-j so they count in opposite directions (one up to i+1, one down from i to 0)
                                                    #All down, to all up.
    #Finds all possible payoffs
    option_values = np.maximum(K - prices[:, N],0) #if K - price is less than 0, sets payoff to zero. (wouldn't use option)
    
    #loop backwards from final price to find best price for now
    for k in range(N-1, -1, -1):
        for l in range(k+1): #possible nodes at step l
            #https://users.physics.ox.ac.uk/~Foot/Phynance/Binomial2013.pdf
            # exp term accounts for risk free return. Rest is standard expected value calculation.
            option_values[l] = np.exp(-r*dt)*((p_u*option_values[l])+((p_d)*option_values[l+1]))
            
    return option_values[0] #returns value at node 0 (the start, ie. how you should price it)

In [22]:
binomial_tree_call(S0,K,T,r,sigma,N=10000)

92.1742377482657

## Monte Carlo

In [24]:
def monte_carlo_call(S0,K,T,r,sigma,N=10000,M=10000):
    """Calculates call price of an European Option using a Monte Carlo Simulation
    using the (risk-neutral) Black Scholes Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility
    N = Number of steps within time T
    M = Number of Simulations"""
    dt = T / N
    payoffs_sum = 0
    
    for i in range(1,M):
        S = np.zeros(N)
        S[0] = S0
        for j in range(1,N):
            #Simulating Geometric Brownian motion https://en.wikipedia.org/wiki/Geometric_Brownian_motion
            Z = np.random.standard_normal() #New Random Number each time
            S[j] = S[j - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
        
        payoffs_sum += np.maximum(S[-1] - K, 0)
        
    avg_payoff = payoffs_sum/M
    price = avg_payoff*np.exp(-r * T) #discount risk free rate
    return price

In [32]:
def analytic_monte_carlo_call(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    M: int = 10000
) -> float:
    """
    Calculates call price of an European Option using a Monte Carlo Simulation
    using the (risk-neutral) Black Scholes Options pricing model (risk-neutral Geometric Brownian Motion (analytic solution)).

    Inputs:
        S0 (float):    Initial stock price.
        K (float):     Strike price.
        T (float):     Time to maturity (in years).
        r (float):     Risk-free interest rate.
        sigma (float): Annual volatility.
        M (int):       Number of Monte Carlo simulations.

    Returns:
        float: Estimated call option value at t=0.
    """

    Z = np.random.standard_normal(M)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * math.sqrt(T) * Z)
    payoffs = np.maximum(ST - K, 0.0)
    expected_payoff = np.mean(payoffs)
    return expected_payoff * math.exp(-r * T) #discounting by risk free rate


In [33]:
def monte_carlo_put(S0,K,T,r,sigma,N=1000,M=1):
    """Calculates put price of an European Option using a Monte Carlo Simulation
    using the (risk-neutral) Black Scholes Options pricing model.
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    sigma = Volatility
    N = Number of steps within time T
    M = Number of simulations"""
    dt = T / N
    payoffs_sum = 0
    
    for i in range(1,M):
        S = np.zeros(N)
        S[0] = S0
        for j in range(1,N):
            #Simulating Geometric Brownian motion https://en.wikipedia.org/wiki/Geometric_Brownian_motion
            Z = np.random.standard_normal() #New Random Number each time
            S[j] = S[j - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
        
        payoffs_sum += np.maximum(K - S[-1], 0)
        
    avg_payoff = payoffs_sum/M
    price = avg_payoff*np.exp(-r * T) #discount risk free rate
    return price

In [34]:
def analytic_monte_carlo_put(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    M: int = 10000
) -> float:
    """
    Calculates put price of an European Option using a Monte Carlo Simulation
    using the (risk-neutral) Black Scholes Options pricing model (risk-neutral Geometric Brownian Motion (analytic solution)).

    Inputs:
        S0 (float):    Initial stock price.
        K (float):     Strike price.
        T (float):     Time to maturity (in years).
        r (float):     Risk-free interest rate.
        sigma (float): Annual volatility.
        M (int):       Number of Monte Carlo simulations.

    Returns:
        float: Estimated call option value at t=0.
    """

    Z = np.random.standard_normal(M)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * math.sqrt(T) * Z)
    payoffs = np.maximum(K - ST, 0.0)
    expected_payoff = np.mean(payoffs)
    return expected_payoff * math.exp(-r * T) #discounting by risk free rate


In [39]:
analytic_monte_carlo_call(S0,K,T,r,sigma)

87.02137862765852

## Heston Model

In [2]:
def volatility_of_volatility(stock):
    """Calculates the annualized volatility of a stock for the 36 preceding 7 day chunks of price data (252 days, one market year),
    and calculates how volatile the volatility is.
    Inputs:
    stock (str): Stock Name (as a string, eg. AAPL)"""
    if type(stock) != str:
        print("Please enter a valid stock code")
        return None
    ticker = Ticker(ticker=stock)
    price = ticker.yahoo_api_price(range='1y', dataGranularity='1d')
    for i in range(0,35):
        price_arr = price['close'][i*7:(i*7)+6]
        pct_chgs = price_arr.pct_change().dropna()  # Calculate % changes 
        std = np.std(pct_chgs)
        volatility = std*np.sqrt(252) #Annualized volatility from 7d of data.
        volatility_arr = np.ones(36)
        volatility_arr[i] = volatility
        
    pct_vol_chgs = volatility_arr.pct_change().dropna()  # Calculate % changes 
    vol_std = np.std(pct_vol_chgs)
    vol_vol = vol_std*np.sqrt(252)
    return vol_vol

In [181]:
def heston_call(S0, K, T, r, v0, kappa, theta, sigma, rho, N, M):        
    """Calculates call price using Heston Options pricing model
    Inputs:
    S0 = Stock Price
    K = Strike Price
    T = Time to maturity in Years
    r = Risk Free Rate
    v0 = Initial Variance (volatility = sqrt(variance))
    kappa = Mean reversion speed of variance
    theta = Long-run variance
    sigma = Volatility of volatility
    rho = Correlation between asset price and volatility changes
    N = Number of steps per run
    M = Number of Runs"""
    dt = T / N
    S = np.full((M, N+1), S0)
    v = np.full((M, N+1), v0)
    
    for i in range(1, N+1):
        Z1 = np.random.standard_normal(M) #Random Zs for each run of the simulation
        Z2 = np.random.standard_normal(M)
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2  # Correlated noise
        
        # Variance # Check if its 0 or i - 1
        v[:, i] = np.maximum(v[:, 0] + kappa * (theta - v[:, i-1]) * dt + sigma * np.sqrt(v[:, i-1] * dt) * Z2, 0)
        
        # Stock price # Check if its 0 or i - 1
        S[:, i] = S[:, 0] * np.exp((r - 0.5 * v[:, i-1]) * dt + np.sqrt(v[:, i-1] * dt) * Z1)
    
    payoff = np.maximum(S[:, -1] - K, 0)
    return np.exp(-r * T) * np.mean(payoff)

## Greeks

In [8]:
import numpy as np
from scipy.stats import norm

def bs_greeks(
    S: float, 
    K: float, 
    T: float, 
    r: float, 
    sigma: float, 
    option_type: str = "call"
) -> dict:
    
    """Calculates the Greeks for a European option using the Black-Scholes model.

    Inputs:
        S (float):     Stock Price.
        K (float):     Strike price.
        T (float):     Time to maturity (in years).
        r (float):     Risk-free rate.
        sigma (float): Volatility (annual).
        option_type (str): "call" or "put"

    Returns:
        dict: Dictionary of Greeks: delta, gamma, vega, theta, rho
        
    Source for formulae:
        https://www.macroption.com/black-scholes-formula/#delta
    
    Note: 252 in theta formula is number of trading days in a year. Makes theta = change in option price per one trading day"""
    
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - (sigma*np.sqrt(T))
    
    pdf_d1 = np.exp(-0.5 * d1**2) / np.sqrt(2 * np.pi) #probability density function (1st deriv)
    
    #not including dividend yield (merton extension)
    if option_type == "call":
        delta = norm.cdf(d1)
        theta = (1/252)*(((-S*sigma*pdf_d1)/(2*np.sqrt(T))) - r*K*np.exp(-r*T)*norm.cdf(d2))
        rho = (1/100)*K*T*np.exp(-r*T)*norm.cdf(d2)
    
    elif option_type == "put":
        delta = norm.cdf(d1) - 1
        theta = (1/252)*(((-S*sigma*pdf_d1)/(2*np.sqrt(T))) + r*K*np.exp(-r*T)*norm.cdf(-d2))
        rho = -(1/100)*K*T*np.exp(-r*T)*norm.cdf(-d2)
    
    else:
        print("Option type must be 'put' or 'call'")
    
    vega = S*pdf_d1*np.sqrt(T)/100
    gamma = pdf_d1/(S*sigma*np.sqrt(T))
    
    return {
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "theta": theta,
        "rho": rho
    }
        

In [15]:
bs_greeks(S0,K,T,r,sigma,'put')['delta']

-0.03105347776105427